Make CSV

In [1]:
import os, glob
import pandas as pd
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter, LowerCaseFilter

In [2]:
wakati = True
target_dir = 'text_dogs'
csv_out = 'text_dogs.csv'

In [3]:
token_filters = [ POSStopFilter(['助詞','助動詞']),
                  LowerCaseFilter(),
                ]
tokenizer = Tokenizer()
analyzer = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

In [4]:
file_pat = '*.txt'
dir_pat = os.path.join(target_dir, '*')

text = []
for d in glob.glob(dir_pat):  # for-loop of directories in target_dir
    if not os.path.isdir(d): continue  # skip if d is not a directory
    category = os.path.basename(d)  # use directory name as the category name
    full_pat = os.path.join(d, file_pat)  # generate full path name for text files
    n_files = 0  # counter for number of files
    for file in glob.glob(full_pat):  # for-loop of text files
        n_files += 1
        with open(file, 'r', encoding='shift-jis') as f:
            lines = f.read().splitlines()  # if you want to omit newline character
            body = ' '.join(lines[2:])  # 1st line: URL, 2nd line: Date, so skip these lines
            if wakati:  # add space between words (wakati-gaki)
                body = ' '.join([t.base_form for t in analyzer.analyze(body)])
            text.append([category, body])
    print(n_files, 'files read for', category)

1 files read for chihuahua
1 files read for golden_retriever
1 files read for shiba


In [5]:
df = pd.DataFrame(text, columns=['category', 'text'])
print(df.shape)
print(df.info())
display(df.head())
display(df.tail())

(3, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  3 non-null      object
 1   text      3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes
None


,category,text
0,chihuahua,19 世紀 半ば アメリカ 品種 改良 進める られる （ この 時期 、 ロング コート ...
1,golden_retriever,1903 年 、 英国 ケネルクラブ ゴールデンレトリーバー 「 フラット コート − ゴー...
2,shiba,温暖 湿潤 気候 強い 。 一般 主人 非常 忠実 、 よそ者 馴れ馴れしい する 、 賢い...


,category,text
0,chihuahua,19 世紀 半ば アメリカ 品種 改良 進める られる （ この 時期 、 ロング コート ...
1,golden_retriever,1903 年 、 英国 ケネルクラブ ゴールデンレトリーバー 「 フラット コート − ゴー...
2,shiba,温暖 湿潤 気候 強い 。 一般 主人 非常 忠実 、 よそ者 馴れ馴れしい する 、 賢い...


In [6]:
df.to_csv(csv_out, index=False)

Doing Bayes

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [8]:
csv_in = 'text_dogs.csv'

# To show all columns and rows  
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [9]:
df = pd.read_csv(csv_in, skiprows=0, delimiter=',', header=0)
display(df.head())
print(df.shape)
print(df.info())
display(df.head())

,category,text
0,chihuahua,19 世紀 半ば アメリカ 品種 改良 進める られる （ この 時期 、 ロング コート ...
1,golden_retriever,1903 年 、 英国 ケネルクラブ ゴールデンレトリーバー 「 フラット コート − ゴー...
2,shiba,温暖 湿潤 気候 強い 。 一般 主人 非常 忠実 、 よそ者 馴れ馴れしい する 、 賢い...


(3, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  3 non-null      object
 1   text      3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes
None


,category,text
0,chihuahua,19 世紀 半ば アメリカ 品種 改良 進める られる （ この 時期 、 ロング コート ...
1,golden_retriever,1903 年 、 英国 ケネルクラブ ゴールデンレトリーバー 「 フラット コート − ゴー...
2,shiba,温暖 湿潤 気候 強い 。 一般 主人 非常 忠実 、 よそ者 馴れ馴れしい する 、 賢い...


In [10]:
X_train = df['text']
y_train = df['category']
print(X_train)
print(y_train)

0    19 世紀 半ば アメリカ 品種 改良 進める られる （ この 時期 、 ロング コート ...
1    1903 年 、 英国 ケネルクラブ ゴールデンレトリーバー 「 フラット コート − ゴー...
2    温暖 湿潤 気候 強い 。 一般 主人 非常 忠実 、 よそ者 馴れ馴れしい する 、 賢い...
Name: text, dtype: object
0           chihuahua
1    golden_retriever
2               shiba
Name: category, dtype: object


In [11]:
X_test = ['短 毛 主人 忠実 勇敢 小型 犬']
print(X_test)

['短 毛 主人 忠実 勇敢 小型 犬']


In [12]:
vectorizer = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')
vectorizer.fit(X_train)
vocab = vectorizer. get_feature_names()
print('Vocabulary size:', len(vocab))
print(vocab)

Vocabulary size: 702
['1', '10', '11', '15', '18', '19', '1903', '1904', '1911', '1938', '1970', '1998', '2', '2000', '2002', '2006', '2017', '24', '29', '3', '34', '37', '4', '40', '5000', '54', '57', '58', '6', '61', '7', '8', 'akc', 'bobo', 'chihuahua', 'cm', 'de', 'dj', 'grca', 'jkc', 'kg', 'あまり', 'ある', 'いう', 'いる', 'おく', 'おる', 'かかわる', 'かつ', 'かむ', 'がかる', 'くる', 'げ', 'こと', 'この', 'これ', 'こん', 'さ', 'さし', 'しかし', 'しつける', 'しまう', 'すぐ', 'する', 'せる', 'その', 'その後', 'それ', 'ただし', 'たち', 'ため', 'つきあい', 'つと', 'てんかん', 'できる', 'とき', 'とも', 'どうし', 'どんどん', 'ない', 'ならでは', 'なる', 'の', 'はしゃぐ', 'また', 'もち', 'もの', 'やく', 'やすい', 'やや', 'やる', 'ゆえ', 'よう', 'よそ者', 'より', 'られる', 'れる', 'わかる', 'アップルドーム', 'アドバイス', 'アメリカ', 'アメリカンケネルクラブ', 'アンダー', 'アーモンド', 'イエロー', 'ウサギ', 'オス', 'オブ', 'カラー', 'ガン', 'ガンドッグ', 'ギジェット', 'クラブ', 'クリーム', 'ケネルクラブ', 'ケース', 'コース', 'コート', 'ゴールデン', 'ゴールデンレトリーバー', 'ショードッグ', 'ジャパンケネルクラブ', 'スペイン', 'スムース', 'ソング', 'タコ', 'タレント', 'タン', 'ダックスフント', 'ダブル', 'チワワ', 'デコ', 'デンタルケア', 'トップコート', 'ドック', 'ハイキング', 'パピヨン', 'ヒット', 'フ

In [13]:
X_train_bow = vectorizer.transform(X_train)
X_test_bow = vectorizer.transform(X_test)
print('X_train_bow:')
print(repr(X_train_bow))
print('X_test_bow:')
print(repr(X_test_bow))

X_train_bow:
<3x702 sparse matrix of type '<class 'numpy.int64'>'
	with 870 stored elements in Compressed Sparse Row format>
X_test_bow:
<1x702 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>


In [14]:
Xbow = pd.DataFrame(X_train_bow.toarray(), 
                    index=y_train, columns=vocab)
display(Xbow)

,1,10,11,15,18,19,1903,1904,1911,1938,1970,1998,2,2000,2002,2006,2017,24,29,3,34,37,4,40,5000,54,57,58,6,61,7,8,akc,bobo,chihuahua,cm,de,dj,grca,jkc,kg,あまり,ある,いう,いる,おく,おる,かかわる,かつ,かむ,がかる,くる,げ,こと,この,これ,こん,さ,さし,しかし,しつける,しまう,すぐ,する,せる,その,その後,それ,ただし,たち,ため,つきあい,つと,てんかん,できる,とき,とも,どうし,どんどん,ない,ならでは,なる,の,はしゃぐ,また,もち,もの,やく,やすい,やや,やる,ゆえ,よう,よそ者,より,られる,れる,わかる,アップルドーム,アドバイス,アメリカ,アメリカンケネルクラブ,アンダー,アーモンド,イエロー,ウサギ,オス,オブ,カラー,ガン,ガンドッグ,ギジェット,クラブ,クリーム,ケネルクラブ,ケース,コース,コート,ゴールデン,ゴールデンレトリーバー,ショードッグ,ジャパンケネルクラブ,スペイン,スムース,ソング,タコ,タレント,タン,ダックスフント,ダブル,チワワ,デコ,デンタルケア,トップコート,ドック,ハイキング,パピヨン,ヒット,フクロウ,フラット,フラットコーテッド,ブラック,ブーム,ベル,ベルク,ベース,ペット,ボディ,ポメラニアン,ポンド,マズル,マン,メス,モンキー,ヨーロッパ,リス,リンゴ,レトリーバー,ロング,ヶ月,一つ,一方,一番,一緒,一般,万,上,上昇,下毛,下部,不全,不足,世紀,並びに,中,丸み,主,主人,乱,事,事故,二,交配,人,人気,人間,今,仔犬,仕事,他,付き,以上,以下,以降,仲間,会員,伸ばす,伸びる,位,低い,低下,住宅,体,体格,体温,体重,体長,体高,作る,作出,使う,使途,例えば,供給,価値,侵入,信頼,個体,個性,偏食,健全,健康,側,傘下,傷付ける,傾向,僅か,働く,優先,先天,光景,入る,全土,全米,公園,兼ね備える,内,再び,冷水,出る,出典,分娩,分担,分野,切る,初めて,別,削る,削除,前述,前部,割,割合,力,劣性,努力,勇敢,動く,動物,化,医師,千,半ば,単独,占める,危険,原因,参照,又は,及び,友好,反る,反面,取れる,取引,古代,可愛い,可愛がる,可能,右,各々,同様,同種,同調,名,向き,吻,呼ぶ,品種,唸る,問題,喜び,噛み付く,四肢,回,国,均一,均整,垂れる,基本,場合,増加,変更,外側,多い,多く,多彩,多様,多種,大,大いに,大きい,大きな,大型,大変,大好き,大胆,失格,女児,女性,好き,好む,好奇,嫌う,子ども,存,季節,孤独,学的,定める,定期,実情,室内,家,家庭,家族,寒い,察知,対峙,対象,寿命,小,小さい,小さな,小型,少ない,尾,尾形,屋内,屋外,山あい,山地,工夫,左巻き,巻き,巻く,希,希少,帯びる,常時,幅広い,年,年代,幼児,幼少,広まる,弱い,張る,強い,強める,弾力,当,当該,形,形成,影響,役割,役目,彼ら,後,後部,心,必要,必須,忍耐,応える,忠実,忠誠,念頭,怠る,性,性格,性質,恒温動物,患う,愛す,愛情,愛想,感染,慣れる,戸外,手伝う,扱う,折れる,抜け落ちる,担う,持ち合わせる,持つ,指定,挙げる,振る舞う,捕らえる,捕食,損ねる,擁す,改良,攻撃,放す,故,敏感,散歩,数,数値,斜視,方,日本,日課,明るい,時,時期,暗い,替わる,最大,月,期,本来,本能,柔らかい,柴犬,根元,検査,極めて,楽しむ,楽天的,概して,様々,機会,機敏,機能,次,次ぐ,欧州,歳,歴史,死亡,死因,毎日,比べる,比率,比較的,毛,毛色,気に入る,気候,気温,気配り,水,求める,沈静,沿う,泉,法則,注意,洋,活動,活発,活躍,派生,浮き上がる,涵養,混同,減少,温和,温暖,湿潤,点,為,無い,無謀,無頓着,爪,特に,特徴,特性,犬,状態,狩り,狩猟,独居,独立,狼,猛禽,猟犬,献身,獣,獰猛,獲物,現在,理想,甘やかす,生える,生まれる,生後,生活,生産,用,由来,甲状腺,留意,番犬,異常,疾患,疾病,病,病気,症,発作,発生,発症,発達,登場,登録,白,的,皮膚,目,目的,直接,真っ直ぐ,眼,瞳,知る,知性,知的,短,短い,研究,硬い,確実,確率,社会,神経,種,種類,穏和,突然,立つ,第,等,管理,米国,約,細やか,細心,組み合わせ,縮れる,繁殖,繊細,繋ぐ,置く,羽毛,習慣,老人,考慮,耳,股関節,肥満,胡麻,胸,能力,脆弱,脱臼,腫れる,腹部,膝蓋骨,臆,自信,臭い,良い,色,若い,英国,茶,茶褐色,菌,行く,行動,表現,被,要,要求,覆う,見た目,見る,見受ける,見知らぬ,規定,視界,親しい,親交,観点,訓練,設立,評判,詰まる,話題,該犬種,認める,語,誰,課題,調整,調査,調節,警戒,豊富,賛美,賞賛,賢い,赤,赤毛,赤金,走り回る,走る,超,身体,近年,追う,追って,通り,連れ出す,進める,遊ぶ,運動,過ぎる,違う,適す,適する,適切,適宜,適応,遺伝,遺伝子,邦題,部分,部屋,重,重んじる,野良,量,鑑みる,長い,長寿,長毛,門,開,関係,関節,防水,雄,集中,雌,難しい,静か,非常,面,項目,頑固,頬,頭,頭数,頭部,額,願う,類,食事,飼い主,飼い犬,飼う,飼育,飾り,餌,馴れ馴れしい,骨格,高い,高額,鳶,鷹,黒,黒色,鼻,８
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
chihuahua,1,0,0,2,1,1,0,1,0,0,1,1,3,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,2,0,1,2,0,1,1,2,1,1,0,1,3,1,11,0,8,0,2,1,1,0,0,0,0,3,1,1,1,6,0,1,0,1,1,21,1,2,1,3,0,1,1,1,0,0,0,0,0,1,1,2,1,8,1,0,4,1,2,2,1,1,0,1,3,0,1,7,12,1,1,1,2,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,5,0,0,1,1,1,2,1,1,1,1,1,0,16,1,1,0,0,1,1,1,1,0,0,1,2,1,1,1,1,0,1,1,1,1,0,0,1,1,1,0,3,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,5,0,0,2,2,5,2,0,1,0,4,0,3,2,0,0,0,0,0,1,0,0,1,1,2,1,1,0,0,1,0,1,0,2,1,1,0,0,2,1,1,1,1,2,0,0,5,1,0,1,1,1,1,1,1,1,0,0,0,0,1,2,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,3,4,1,1,2,0,0,2,0,1,0,0,0,1,1,0,1,0,1,1,2,0,0,2,1,0,1,0,1,2,1,1,1,0,1,0,0,1,1,0,0,1,0,0,1,1,5,2,0,1,1,2,0,4,2,1,1,0,1,1,0,2,1,1,1,0,0,1,0,0,1,0,0,2,0,1,0,0,1,0,1,1,1,1,4,1,5,1,0,0,0,2,0,0,0,0,0,0,0,1,1,0,0,5,2,0,1,1,2,1,2,0,0,0,0,1,0,0,0,0,1,0,0,3,3,1,0,1,0,1,0,1,4,0,1,1,0,0,1,1,1,0,0,0,0,0,0,1,0,3,0,1,1,0,0,0,0,2,0,1,0,1,1,0,1,1,0,4,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,2,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,2,1,0,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,1,1,0,0,

In [15]:
model = MultinomialNB(alpha=1.0)
model.fit(X_train_bow, y_train)
print(model.classes_)
train_score = model.score(X_train_bow, y_train)
print('Train accuracy:', train_score)

['chihuahua' 'golden_retriever' 'shiba']
Train accuracy: 1.0


In [16]:
proba = model.predict_proba(X_test_bow)
results = pd.DataFrame(proba, columns=model.classes_)
print('Prediction:')
display(results)
print(model.predict(X_test_bow))

Prediction:


,chihuahua,golden_retriever,shiba
0,0.042454,0.011998,0.945547


['shiba']
